In [4]:
from sklearn import svm

from kan import *
import matplotlib.pyplot as plt
import pandas as pd
import argparse
import numpy as np
import os
from copy import deepcopy
from tqdm.auto import tqdm
from dataloaders.dataloader import FCMatrixDataset
from utils import balanced_random_split_v2

from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data.dataset import random_split
from torch.utils.data.sampler import SubsetRandomSampler

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix
from fasterkan.fasterkan import FasterKAN



import torch
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

In [10]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=99)

mrmr_features = np.array([1140, 536, 223, 907, 1449, 499, 1293, 45, 135, 1440, 879, 1384, 1210, 1316, 122, 22, 492, 638, 765, 1027, 1464, 501, 1462, 395, 26, 1079, 70, 425, 1403, 1409, 1318, 886, 1459, 1448, 939, 1163, 547, 10, 413, 676, 131, 216, 942, 1136, 1386, 232, 1455, 1337, 814, 139, 392, 1376, 1382, 471, 656]
    )
# mrmr_features=None

ds = "data/csv/severe_rds.csv"
# ds = "data/csv/balanced_sex_classification.csv"
data_dir = "data/fetched/25751/raw"
udi = "25751"
dataset = FCMatrixDataset(ds, data_dir, udi, mapping=None, mrmr=mrmr_features)
labels = np.genfromtxt(ds)
eids = labels[:,0]
labels = labels[:,1]
eval_metrics = np.zeros((skf.n_splits, 2))

# print(dataset[0][0].shape)



# total_size = len(ds)
# train_size = int(0.8 * total_size)
# val_size = int(0.1 * total_size)
# test_size = total_size - train_size - val_size

# train, val, test = balanced_random_split_v2(ds, [train_size, val_size, test_size], num_classes=2)

# train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)



In [15]:
np.random.seed(41)

for n_fold, (train_val, test) in enumerate(skf.split(labels, labels)):
    print
    clf = svm.SVC(kernel='poly', degree=5, class_weight='balanced')
    clf = svm.SVC(kernel='rbf')
    # clf = svm.SVC(kernel='linear', C=0.01, gamma=1.0, class_weight='balanced')
    # model = KAN([1485, 64, 2])

    train_val_dataset, test_dataset = Subset(dataset, train_val), Subset(dataset, test)
    train_val_labels = labels[train_val]
    train_val_index = np.arange(len(train_val_dataset))

    train, val, _, _ = train_test_split(train_val_index, train_val_labels, test_size=0.11, shuffle=True, stratify=train_val_labels)
    train_dataset, val_dataset = Subset(train_val_dataset, train), Subset(train_val_dataset, val)
    
    def seperate_subset(subset):
        input = [np.array(subset[i][0]) for i in range(len(subset))]
        input = np.array(input)
        label = [subset[i][1] for i in range(len(subset))]
        label = np.array(label)
        return input, label

    train_input, train_label = seperate_subset(train_dataset)
    val_input, val_label = seperate_subset(val_dataset)
    test_input, test_label = seperate_subset(test_dataset)
   
    clf.fit(train_input, train_label)
    val_pred = clf.predict(val_input)
    print(val_pred)
    print(val_label)
    val_acc = np.mean(val_pred == val_label)
    test_pred = clf.predict(test_input)
    test_acc = np.mean(test_pred == test_label)


    # test_sen, test_spe, test_acc, _ = test(test_dataset)

    print('Fold %i' % (n_fold + 1), 'Val Acc: %.4f' % val_acc, 'Test Acc: %.4f' % test_acc)
    

    eval_metrics[n_fold, 0] = val_acc
    eval_metrics[n_fold, 1] = test_acc


eval_df = pd.DataFrame(eval_metrics)
eval_df.columns = ['ACC','test_acc']
eval_df.index = ['Fold_%02i' % (i + 1) for i in range(skf.n_splits)]
print(eval_df)
# print('Average Sensitivity: %.4f±%.4f' % (eval_metrics[:, 0].mean(), eval_metrics[:, 0].std()))
# print('Average Specificity: %.4f±%.4f' % (eval_metrics[:, 1].mean(), eval_metrics[:, 1].std()))
print('Average Accuracy: %.4f±%.4f' % (eval_metrics[:, 0].mean(), eval_metrics[:, 0].std()))
print('Average Accuracy: %.4f±%.4f' % (eval_metrics[:, 0].mean(), eval_metrics[:, 1].std()))


[1 1 0 1 1 1 1 1 0 0 1 0 1 1 1 0 0 1 1 0 0 0 1 0]
[1 0 1 1 0 1 1 1 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0]
Fold 1 Val Acc: 0.7500 Test Acc: 0.6667
[1 1 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 1 0 0 1 1 0]
[1 1 0 0 1 1 1 1 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0]
Fold 2 Val Acc: 0.7500 Test Acc: 0.6250
[0 1 1 0 0 0 1 0 1 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0]
[1 1 1 0 1 0 1 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 1 0]
Fold 3 Val Acc: 0.7917 Test Acc: 0.9167
[1 1 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1]
[1 1 1 1 0 1 0 0 0 1 1 0 1 1 0 1 0 0 0 1 0 0 0 1]
Fold 4 Val Acc: 0.8333 Test Acc: 0.8333
[1 1 0 1 0 0 0 1 1 0 1 1 0 1 0 0 0 0 1 1 0 1 0 0]
[1 1 0 1 0 0 0 1 1 0 1 1 0 1 0 0 0 0 1 1 0 1 0 1]
Fold 5 Val Acc: 0.9583 Test Acc: 0.8750
[0 0 0 1 1 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0]
[0 1 0 1 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 0 0 0 1]
Fold 6 Val Acc: 0.6250 Test Acc: 0.8333
[0 0 0 0 1 1 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0 0 0 1]
[0 1 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1]
Fold 7 Val Acc: 0.8750 Test Acc: 0.8696
[1 1 0 0 1 0 0 1 1 0